In [5]:
import itertools

In [ ]:
xs = [[0, 0, 1, 0],
      [0, 1, 0, 0],
      [0, 0, 1, 1],
      [1, 0, 0, 1],
      [0, 1, 1, 0],
      [1, 1, 0, 0],
      [0, 1, 0, 1]]

ys = [0, 0, 1, 1, 0, 0, 0]

In [26]:
def eval_conjunctive_rule(x, r):
    res = 1 # initialize rule with 1 (neutral number for products)
    for i in r:
        res *= x[i] # do and with the element selected by r
    return res

def make_conjunctive_rule_readable(r):
    if r:
        res = ''
        for i in r[:-1]:
            res += 'x_' + str((i+1)) + ' ∧ '

        res += 'x_' + str((r[-1]+1))
    else:
        res = '1'
    return res

In [27]:
def generate_all_conjunctive_rules(n):
    res = []
    for size in range(n+1):
        for rule in itertools.combinations(range(n), size):
            res.append(rule)
    return res

In [32]:
def count_mistakes(hat_ys, ys):
    res = 0
    for i in range(len(hat_ys)):
        if hat_ys[i] != ys[i]:
            res += 1
    return res

In [33]:
conjunctive_rules = generate_all_conjunctive_rules(4)

conjunctive_rules

[(),
 (0,),
 (1,),
 (2,),
 (3,),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 2),
 (1, 3),
 (2, 3),
 (0, 1, 2),
 (0, 1, 3),
 (0, 2, 3),
 (1, 2, 3),
 (0, 1, 2, 3)]

In [34]:
for rule in conjunctive_rules:
    print(make_conjunctive_rule_readable(rule))

1
x_1
x_2
x_3
x_4
x_1 ∧ x_2
x_1 ∧ x_3
x_1 ∧ x_4
x_2 ∧ x_3
x_2 ∧ x_4
x_3 ∧ x_4
x_1 ∧ x_2 ∧ x_3
x_1 ∧ x_2 ∧ x_4
x_1 ∧ x_3 ∧ x_4
x_2 ∧ x_3 ∧ x_4
x_1 ∧ x_2 ∧ x_3 ∧ x_4


In [35]:
for r in conjunctive_rules:
    hat_ys = []
    for x in xs:
        hat_ys.append(eval_conjunctive_rule(x, r))

    mistakes = count_mistakes(hat_ys, ys)
    print('The rule', make_conjunctive_rule_readable(r), 'has generated', mistakes, 'mistakes')

The rule 1  has generated 5 mistakes
The rule x_1  has generated 2 mistakes
The rule x_2  has generated 6 mistakes
The rule x_3  has generated 3 mistakes
The rule x_4  has generated 1 mistakes
The rule x_1 ∧ x_2  has generated 3 mistakes
The rule x_1 ∧ x_3  has generated 2 mistakes
The rule x_1 ∧ x_4  has generated 1 mistakes
The rule x_2 ∧ x_3  has generated 3 mistakes
The rule x_2 ∧ x_4  has generated 3 mistakes
The rule x_3 ∧ x_4  has generated 1 mistakes
The rule x_1 ∧ x_2 ∧ x_3  has generated 2 mistakes
The rule x_1 ∧ x_2 ∧ x_4  has generated 2 mistakes
The rule x_1 ∧ x_3 ∧ x_4  has generated 2 mistakes
The rule x_2 ∧ x_3 ∧ x_4  has generated 2 mistakes
The rule x_1 ∧ x_2 ∧ x_3 ∧ x_4  has generated 2 mistakes


In [56]:
def eval_m_of_n_rule(x, r):
    count = 0 # initialize rule with 0 (neutral number for summation)
    m, variables = r
    for i in variables:
        count += x[i] # count how many variables match
    if count >= m:
        return 1
    return 0

def make_m_of_n_rule_readable(r):
    m, variables = r
    res = str(m) + '-of-{'
    for i in variables[:-1]:
        res += 'x_' + str(i+1) + ', '
    res += 'x_' + str(variables[-1]+1) + '}'
    return res

In [53]:
def generate_all_m_of_n_rules(n):
    res = []
    for size in range(n+1):
        for variables in itertools.combinations(range(n), size):
            for m in range(len(variables)):
                res.append((m + 1, variables))
    return res

m_of_n_rules = generate_all_m_of_n_rules(4)

m_of_n_rules

[(1, (0,)),
 (1, (1,)),
 (1, (2,)),
 (1, (3,)),
 (1, (0, 1)),
 (2, (0, 1)),
 (1, (0, 2)),
 (2, (0, 2)),
 (1, (0, 3)),
 (2, (0, 3)),
 (1, (1, 2)),
 (2, (1, 2)),
 (1, (1, 3)),
 (2, (1, 3)),
 (1, (2, 3)),
 (2, (2, 3)),
 (1, (0, 1, 2)),
 (2, (0, 1, 2)),
 (3, (0, 1, 2)),
 (1, (0, 1, 3)),
 (2, (0, 1, 3)),
 (3, (0, 1, 3)),
 (1, (0, 2, 3)),
 (2, (0, 2, 3)),
 (3, (0, 2, 3)),
 (1, (1, 2, 3)),
 (2, (1, 2, 3)),
 (3, (1, 2, 3)),
 (1, (0, 1, 2, 3)),
 (2, (0, 1, 2, 3)),
 (3, (0, 1, 2, 3)),
 (4, (0, 1, 2, 3))]

In [57]:
for rule in m_of_n_rules:
    print(make_m_of_n_rule_readable(rule))

1-of-{x_1}
1-of-{x_2}
1-of-{x_3}
1-of-{x_4}
1-of-{x_1, x_2}
2-of-{x_1, x_2}
1-of-{x_1, x_3}
2-of-{x_1, x_3}
1-of-{x_1, x_4}
2-of-{x_1, x_4}
1-of-{x_2, x_3}
2-of-{x_2, x_3}
1-of-{x_2, x_4}
2-of-{x_2, x_4}
1-of-{x_3, x_4}
2-of-{x_3, x_4}
1-of-{x_1, x_2, x_3}
2-of-{x_1, x_2, x_3}
3-of-{x_1, x_2, x_3}
1-of-{x_1, x_2, x_4}
2-of-{x_1, x_2, x_4}
3-of-{x_1, x_2, x_4}
1-of-{x_1, x_3, x_4}
2-of-{x_1, x_3, x_4}
3-of-{x_1, x_3, x_4}
1-of-{x_2, x_3, x_4}
2-of-{x_2, x_3, x_4}
3-of-{x_2, x_3, x_4}
1-of-{x_1, x_2, x_3, x_4}
2-of-{x_1, x_2, x_3, x_4}
3-of-{x_1, x_2, x_3, x_4}
4-of-{x_1, x_2, x_3, x_4}


In [59]:
for r in m_of_n_rules:
    hat_ys = []
    for x in xs:
        hat_ys.append(eval_m_of_n_rule(x, r))

    mistakes = count_mistakes(hat_ys, ys)
    print('The rule', make_m_of_n_rule_readable(r), 'has generated', mistakes, 'mistakes')

The rule 1-of-{x_1} has generated 2 mistakes
The rule 1-of-{x_2} has generated 6 mistakes
The rule 1-of-{x_3} has generated 3 mistakes
The rule 1-of-{x_4} has generated 1 mistakes
The rule 1-of-{x_1, x_2} has generated 5 mistakes
The rule 2-of-{x_1, x_2} has generated 3 mistakes
The rule 1-of-{x_1, x_3} has generated 3 mistakes
The rule 2-of-{x_1, x_3} has generated 2 mistakes
The rule 1-of-{x_1, x_4} has generated 2 mistakes
The rule 2-of-{x_1, x_4} has generated 1 mistakes
The rule 1-of-{x_2, x_3} has generated 6 mistakes
The rule 2-of-{x_2, x_3} has generated 3 mistakes
The rule 1-of-{x_2, x_4} has generated 4 mistakes
The rule 2-of-{x_2, x_4} has generated 3 mistakes
The rule 1-of-{x_3, x_4} has generated 3 mistakes
The rule 2-of-{x_3, x_4} has generated 1 mistakes
The rule 1-of-{x_1, x_2, x_3} has generated 5 mistakes
The rule 2-of-{x_1, x_2, x_3} has generated 4 mistakes
The rule 3-of-{x_1, x_2, x_3} has generated 2 mistakes
The rule 1-of-{x_1, x_2, x_4} has generated 4 mistakes
